In [17]:
import pandas as pd
import numpy as np
from scipy import stats
import collections
import math
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import collections
import imblearn.under_sampling
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from keras.models import Sequential
from keras.layers import Activation, Dense, Conv1D
from keras import losses
from tensorflow.keras import optimizers
import tensorflow as tf

import matplotlib.pyplot as plt



In [51]:
df = pd.read_csv('../data/stroke_data_cleanedJB.csv')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,never smoked,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,never smoked,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,children,Rural,161.28,19.1,never smoked,0


Convert smoking status to cardinal, making sure to keep the NaN values as null:

In [52]:
df.loc[df["smoking_status"] == "never smoked", "smoking_status_cardinal"] = 0
df.loc[df["smoking_status"] == "formerly smoked", "smoking_status_cardinal"] = 1
df.loc[df["smoking_status"] == "smokes", "smoking_status_cardinal"] = 2

In [54]:
df[['smoking_status', 'smoking_status_cardinal']].head(20)

,smoking_status,smoking_status_cardinal
0,never smoked,0.0
1,never smoked,0.0
2,never smoked,0.0
3,formerly smoked,1.0
4,never smoked,0.0
5,NaN,NaN
6,formerly smoked,1.0
7,never smoked,0.0
8,smokes,2.0
9,never smoked,0.0


Convert all the binary columns to 0/1:

In [55]:
df.loc[df["gender"] == "Male", "gender"] = 0
df.loc[df["gender"] == "Female", "gender"] = 1

In [56]:
df.loc[df["ever_married"] == "No", "ever_married"] = 0
df.loc[df["ever_married"] == "Yes", "ever_married"] = 1

In [57]:
df.loc[df["Residence_type"] == "Rural", "Residence_type"] = 0
df.loc[df["Residence_type"] == "Urban", "Residence_type"] = 1

For work_type, since there are 5 categories, I converted them to one-hot encoding:

In [58]:
df_onehot_worktype = pd.get_dummies(df['work_type'])
df_onehot_worktype.columns = ["work_type_"+col for col in df_onehot_worktype.columns]
df_onehot_worktype

,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children
0,0,0,0,0,1
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,0,0,1
...,...,...,...,...,...
43384,0,0,0,0,1
43385,1,0,0,0,0
43386,0,0,1,0,0
43387,0,0,1,0,0


In [59]:
df = pd.concat([df, df_onehot_worktype], axis=1)

Remove previous columns:

In [60]:
df_imputation = df.drop(columns=["smoking_status", "work_type", "id"])

In [61]:
df_imputation

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,smoking_status_cardinal,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children
0,0,3.0,0,0,0,0,95.12,18.0,0,0.0,0,0,0,0,1
1,0,58.0,1,0,1,1,87.96,39.2,0,0.0,0,0,1,0,0
2,1,8.0,0,0,0,1,110.89,17.6,0,0.0,0,0,1,0,0
3,1,70.0,0,0,1,0,69.04,35.9,0,1.0,0,0,1,0,0
4,0,14.0,0,0,0,0,161.28,19.1,0,0.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43384,1,10.0,0,0,0,1,58.64,20.4,0,0.0,0,0,0,0,1
43385,1,56.0,0,0,1,1,213.61,55.4,0,1.0,1,0,0,0,0
43386,1,82.0,1,0,1,1,91.94,28.9,0,1.0,0,0,1,0,0
43387,0,40.0,0,0,1,1,99.16,33.2,0,0.0,0,0,1,0,0


Count total number of strokes, and save 20% for the testing dataset (and then save the same number of non-stroke rows). I chose only rows with non-NaN rows, so as not to impute anything in the test data (which could skew our results):

In [62]:
sum(df_imputation["stroke"] == 1)

783

In [63]:
num_stroke_test = round(sum(df_imputation["stroke"] == 1) / 5)

In [64]:
num_stroke_test

157

First shuffle the dataframe:

In [65]:
df_imputation = df_imputation.sample(frac=1, random_state=1).reset_index(drop=True)

Generate test dataset before imputation, taking only non NaN values:

In [66]:
index_test_stroke = df_imputation.dropna().loc[df_imputation.dropna()["stroke"] == 1,:].iloc[0:num_stroke_test,:].index
index_test_nonstroke = df_imputation.dropna().loc[df_imputation.dropna()["stroke"] == 0,:].iloc[0:num_stroke_test,:].index

In [67]:
test_dataset = pd.concat([df_imputation.loc[index_test_stroke,:], df_imputation.loc[index_test_nonstroke,:]])
test_dataset

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,smoking_status_cardinal,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children
10,0,58.00,0,1,1,0,63.26,29.6,1,0.0,0,0,1,0,0
53,1,78.00,0,0,1,1,81.59,32.3,1,0.0,0,0,1,0,0
59,0,59.00,0,0,1,0,91.59,31.9,1,2.0,0,0,1,0,0
240,1,68.00,0,0,0,1,231.96,36.7,1,2.0,0,0,0,1,0
326,0,79.00,0,0,1,1,87.06,24.3,1,1.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,24.00,0,0,0,0,68.13,20.2,0,0.0,0,0,1,0,0
196,1,76.00,0,0,1,1,106.84,27.0,0,1.0,0,0,0,1,0
197,1,0.24,0,0,0,0,89.24,13.8,0,0.0,0,0,0,0,1
198,1,33.00,0,0,0,0,114.27,25.3,0,0.0,1,0,0,0,0


Remove training samples from dataset:

In [68]:
train_dataset = df_imputation.drop(index=index_test_stroke)
train_dataset = train_dataset.drop(index=index_test_nonstroke)
train_dataset

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,smoking_status_cardinal,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children
3,1,65.0,1,0,1,0,81.70,29.5,0,NaN,0,0,0,1,0
17,0,17.0,0,0,0,1,82.57,28.1,0,NaN,0,0,1,0,0
18,1,65.0,0,0,1,0,97.54,24.4,0,NaN,1,0,0,0,0
27,1,57.0,0,0,1,1,139.73,NaN,1,2.0,0,0,0,1,0
30,1,68.0,0,1,1,1,235.75,32.2,0,NaN,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43384,0,3.0,0,0,0,1,58.50,21.0,0,0.0,0,0,0,0,1
43385,1,0.4,0,0,0,1,105.52,13.6,0,0.0,0,0,0,0,1
43386,0,60.0,0,0,1,1,243.65,19.4,0,2.0,0,0,1,0,0
43387,0,46.0,0,0,1,1,62.51,24.1,0,NaN,0,0,0,1,0


Impute missing values (in smoking_status and bmi):

In [69]:
imp = KNNImputer(n_neighbors=5)
train_dataset_imputed = imp.fit_transform(train_dataset)

In [70]:
df_train_dataset_imputed = pd.DataFrame(train_dataset_imputed, columns=train_dataset.columns)
df_train_dataset_imputed

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,smoking_status_cardinal,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children
0,1.0,65.0,1.0,0.0,1.0,0.0,81.70,29.50,0.0,0.6,0.0,0.0,0.0,1.0,0.0
1,0.0,17.0,0.0,0.0,0.0,1.0,82.57,28.10,0.0,0.6,0.0,0.0,1.0,0.0,0.0
2,1.0,65.0,0.0,0.0,1.0,0.0,97.54,24.40,0.0,0.8,1.0,0.0,0.0,0.0,0.0
3,1.0,57.0,0.0,0.0,1.0,1.0,139.73,36.26,1.0,2.0,0.0,0.0,0.0,1.0,0.0
4,1.0,68.0,0.0,1.0,1.0,1.0,235.75,32.20,0.0,0.2,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43070,0.0,3.0,0.0,0.0,0.0,1.0,58.50,21.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43071,1.0,0.4,0.0,0.0,0.0,1.0,105.52,13.60,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43072,0.0,60.0,0.0,0.0,1.0,1.0,243.65,19.40,0.0,2.0,0.0,0.0,1.0,0.0,0.0
43073,0.0,46.0,0.0,0.0,1.0,1.0,62.51,24.10,0.0,0.6,0.0,0.0,0.0,1.0,0.0


In [71]:
df_train_dataset_imputed["gender"].dtype

dtype('float64')

Since imputation for cardinal smoking status gives non-cardinal values, need to round to integer:

In [72]:
df_train_dataset_imputed.loc[:,"smoking_status_cardinal"] = round(df_train_dataset_imputed.loc[:,"smoking_status_cardinal"])
df_train_dataset_imputed.loc[:,"smoking_status_cardinal"] = df_train_dataset_imputed.loc[:,"smoking_status_cardinal"].astype('float64')
df_train_dataset_imputed

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,smoking_status_cardinal,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children
0,1.0,65.0,1.0,0.0,1.0,0.0,81.70,29.50,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,17.0,0.0,0.0,0.0,1.0,82.57,28.10,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,65.0,0.0,0.0,1.0,0.0,97.54,24.40,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1.0,57.0,0.0,0.0,1.0,1.0,139.73,36.26,1.0,2.0,0.0,0.0,0.0,1.0,0.0
4,1.0,68.0,0.0,1.0,1.0,1.0,235.75,32.20,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43070,0.0,3.0,0.0,0.0,0.0,1.0,58.50,21.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43071,1.0,0.4,0.0,0.0,0.0,1.0,105.52,13.60,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43072,0.0,60.0,0.0,0.0,1.0,1.0,243.65,19.40,0.0,2.0,0.0,0.0,1.0,0.0,0.0
43073,0.0,46.0,0.0,0.0,1.0,1.0,62.51,24.10,0.0,1.0,0.0,0.0,0.0,1.0,0.0


Oversample the training data so that we have an equal number of strokes/non-strokes:

In [73]:
X_train = df_train_dataset_imputed.drop(columns="stroke")
y_train = df_train_dataset_imputed.loc[:, "stroke"]
rus = RandomOverSampler(random_state=1)
X_train_oversampled, y_train_oversampled = rus.fit_resample(X_train, y_train)
print('Ratio of strokes over the total number of train samples is:', y_train_oversampled.sum()/y_train_oversampled.shape[0])

Ratio of strokes over the total number of train samples is: 0.5


In [75]:
train_imputed = pd.concat([X_train_oversampled, y_train_oversampled], axis=1)
train_imputed.shape

(84898, 15)

In [76]:
train_imputed_notoversampled = pd.concat([X_train, y_train], axis=1)
train_imputed_notoversampled.shape

(43075, 15)

In [66]:
test_dataset.shape

(314, 15)

In [77]:
train_imputed.to_csv('../data/train_allnumeric_oversampled_imputed_JB.csv', index=False)
test_dataset.to_csv('../data/test_allnumeric_JB.csv', index=False)


Reconvert to str values for Rshiny:

In [85]:
df_reformat_train_imputed = train_imputed_notoversampled
for col in ["hypertension", "heart_disease", "ever_married", "stroke"]:
    df_reformat_train_imputed.loc[df_reformat_train_imputed[col] == 0, col+"_reformat"] = "No"
    df_reformat_train_imputed.loc[df_reformat_train_imputed[col] == 1, col+"_reformat"] = "Yes"

df_reformat_train_imputed.loc[df_reformat_train_imputed["gender"] == 0, "gender_reformat"] = "Male"
df_reformat_train_imputed.loc[df_reformat_train_imputed["gender"] == 1, "gender_reformat"] = "Female"

df_reformat_train_imputed.loc[df_reformat_train_imputed["Residence_type"] == 0, "Residence_type_reformat"] = "Rural"
df_reformat_train_imputed.loc[df_reformat_train_imputed["Residence_type"] == 1, "Residence_type_reformat"] = "Urban"

df_reformat_train_imputed.loc[df_reformat_train_imputed["work_type_Govt_job"] == 1, "work_type_reformat"] = "Government job"
df_reformat_train_imputed.loc[df_reformat_train_imputed["work_type_Never_worked"] == 1, "work_type_reformat"] = "Never worked"
df_reformat_train_imputed.loc[df_reformat_train_imputed["work_type_Private"] == 1, "work_type_reformat"] = "Private"
df_reformat_train_imputed.loc[df_reformat_train_imputed["work_type_Self-employed"] == 1, "work_type_reformat"] = "Self-employed"
df_reformat_train_imputed.loc[df_reformat_train_imputed["work_type_children"] == 1, "work_type_reformat"] = "Child"

df_reformat_train_imputed.loc[df_reformat_train_imputed["smoking_status_cardinal"] == 0, "smoking_status_reformat"] = "Never smoker"
df_reformat_train_imputed.loc[df_reformat_train_imputed["smoking_status_cardinal"] == 1, "smoking_status_reformat"] = "Former smoker"
df_reformat_train_imputed.loc[df_reformat_train_imputed["smoking_status_cardinal"] == 2, "smoking_status_reformat"] = "Current smoker"
df_reformat_train_imputed = df_reformat_train_imputed.rename(columns={"smoking_status_cardinal": "smoking_status"})
df_reformat_train_imputed

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,smoking_status,work_type_Govt_job,...,work_type_children,stroke,hypertension_reformat,heart_disease_reformat,ever_married_reformat,stroke_reformat,gender_reformat,Residence_type_reformat,work_type_reformat,smoking_status_reformat
0,1.0,65.0,1.0,0.0,1.0,0.0,81.70,29.50,1.0,0.0,...,0.0,0.0,Yes,No,Yes,No,Female,Rural,Self-employed,Former smoker
1,0.0,17.0,0.0,0.0,0.0,1.0,82.57,28.10,1.0,0.0,...,0.0,0.0,No,No,No,No,Male,Urban,Private,Former smoker
2,1.0,65.0,0.0,0.0,1.0,0.0,97.54,24.40,1.0,1.0,...,0.0,0.0,No,No,Yes,No,Female,Rural,Government job,Former smoker
3,1.0,57.0,0.0,0.0,1.0,1.0,139.73,36.26,2.0,0.0,...,0.0,1.0,No,No,Yes,Yes,Female,Urban,Self-employed,Current smoker
4,1.0,68.0,0.0,1.0,1.0,1.0,235.75,32.20,0.0,0.0,...,0.0,0.0,No,Yes,Yes,No,Female,Urban,Private,Never smoker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43070,0.0,3.0,0.0,0.0,0.0,1.0,58.50,21.00,0.0,0.0,...,1.0,0.0,No,No,No,No,Male,Urban,Child,Never smoker
43071,1.0,0.4,0.0,0.0,0.0,1.0,105.52,13.60,0.0,0.0,...,1.0,0.0,No,No,No,No,Female,Urban,Child,Never smoker
43072,0.0,60.0,0.0,0.0,1.0,1.0,243.65,19.40,2.0,0.0,...,0.0,0.0,No,No,Yes,No,Male,Urban,Private,Current smoker
43073,0.0,46.0,0.0,0.0,1.0,1.0,62.51,24.10,1.0,0.0,...,0.0,0.0,No,No,Yes,No,Male,Urban,Self-employed,Former smoker


In [86]:
df_reformat_train_imputed.to_csv('../data/train_allnumeric_notoversampled_imputed_reformatted_JB.csv', index=False)

In [ ]:
df_reformat_test = test_dataset
for col in ["hypertension", "heart_disease", "ever_married", "stroke"]:
    df_reformat_test.loc[df_reformat_test[col] == 0, col+"_reformat"] = "No"
    df_reformat_test.loc[df_reformat_test[col] == 1, col+"_reformat"] = "Yes"

df_reformat_test.loc[df_reformat_test["gender"] == 0, "gender_reformat"] = "Male"
df_reformat_test.loc[df_reformat_test["gender"] == 1, "gender_reformat"] = "Female"

df_reformat_test.loc[df_reformat_test["Residence_type"] == 0, "Residence_type_reformat"] = "Rural"
df_reformat_test.loc[df_reformat_test["Residence_type"] == 1, "Residence_type_reformat"] = "Urban"

df_reformat_test.loc[df_reformat_test["work_type_Govt_job"] == 1, "work_type_reformat"] = "Government job"
df_reformat_test.loc[df_reformat_test["work_type_Never_worked"] == 1, "work_type_reformat"] = "Never worked"
df_reformat_test.loc[df_reformat_test["work_type_Private"] == 1, "work_type_reformat"] = "Private"
df_reformat_test.loc[df_reformat_test["work_type_Self-employed"] == 1, "work_type_reformat"] = "Self-employed"
df_reformat_test.loc[df_reformat_test["work_type_children"] == 1, "work_type_reformat"] = "Child"

df_reformat_test.loc[df_reformat_test["smoking_status_cardinal"] == 0, "smoking_status_reformat"] = "Never smoker"
df_reformat_test.loc[df_reformat_test["smoking_status_cardinal"] == 1, "smoking_status_reformat"] = "Former smoker"
df_reformat_test.loc[df_reformat_test["smoking_status_cardinal"] == 2, "smoking_status_reformat"] = "Current smoker"
df_reformat_test.rename(columns={"smoking_status_cardinal"})
df_reformat_test